# Lollipop Plot

A lollipop plot displays each element of a dataset as a segment and a circle. It is usually combined with the `count` stat, and is especially useful when you have several bars of the same height.

1. [Parameters `size`, `stroke` and `linewidth`](#stroke)

2. [Parameter `fatten`](#fatten)

3. [Horizontal Sticks](#direction)

4. [Sloped Baseline](#slope)

5. [Parameter `stat`](#stat)

6. [Lollipops in Marginal Layer](#ggmarginal)

7. [Lollipops and a Regression Line](#slope_and_intercept)


In [1]:
%useLatestDescriptors
%use lets-plot

In [2]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.4.8.0. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.4.4.1.

In [3]:
%use dataframe(0.13.0)

In [4]:
import org.jetbrains.letsPlot.core.plot.base.stat.regression.LinearRegression
import kotlin.random.Random

In [5]:
val rnd = Random(0)
val data = mapOf(
    "x" to (-15..14).toList(),
    "y" to (0..29).map { rnd.nextDouble(1.0, 5.0) },
    "sugar" to (150..179).toList()
)

In [6]:
letsPlot(data) { x = "x"; y = "y" } + geomLollipop() + ggsize(600, 200)

<a id="stroke"></a>

#### 1. Parameters `size`, `stroke` and `linewidth`

In [7]:
val plots = listOf(
    letsPlot(data) { x = "x"; y = "y"; size = "sugar" } + 
        geomLollipop() + ggtitle("variable 'size'"),
    letsPlot(data) { x = "x"; y = "y"; size = "sugar"; stroke = "sugar" } + 
        geomLollipop() + ggtitle("variable 'size' and 'stroke'"),
    letsPlot(data) { x = "x"; y = "y"; size = "sugar"; linewidth = "sugar" } + 
        geomLollipop() + ggtitle("variable 'size' and 'linewidth'")
)
    
gggrid(plots, ncol=1) + ggsize(800, 800)

<a id="fatten"></a>

#### 2. Parameter `fatten`

In [8]:
gggrid(
    listOf(
        letsPlot(data) { x = "x"; y = "y"} + geomLollipop() + ggtitle("fatten=2.5 (default)"),
        letsPlot(data) { x = "x"; y = "y"} + geomLollipop(fatten = 5) + ggtitle("fatten=5"),
    )
)

<a id="direction"></a>

#### 3. Horizontal Sticks

In [9]:
letsPlot(data) { x = "y"; y = "x" } + geomLollipop(dir = "h")

<a id="slope"></a>

#### 4. Sloped Baseline

In [10]:
val slope = 0.5
val intercept = 1

val abline = letsPlot(data) { x = "x"; y = "y" } +
    geomABLine(intercept = intercept, slope = slope, color = "black", linetype = "dotted", size = 1.5) + 
    coordFixed(ylim = -12 to 12)
    
gggrid(
    listOf(
        abline + geomLollipop(intercept = intercept, slope = slope, shape = 21) + ggtitle("dir='v' (default)"),
        abline + geomLollipop(intercept = intercept, slope = slope, shape = 21, dir = "h") + ggtitle("dir='h'"),
        abline + geomLollipop(intercept = intercept, slope = slope, shape = 21, dir = "s") + ggtitle("dir='s'"),
    )
)

<a id="stat"></a>

#### 5. Parameter `stat`

In [11]:
val df = DataFrame.readCSV("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/mpg.csv")
df.head(3)

untitled,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.800000,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.800000,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.000000,2008,4,manual(m6),f,20,31,p,compact


In [12]:
gggrid(
    listOf(
        letsPlot(df.toMap()) { x = "class" } + geomLollipop(stat = Stat.count()) + ggtitle("stat='count'"),
        letsPlot(df.toMap()) { x = "hwy" } + geomLollipop(stat = Stat.bin()) + ggtitle("stat='bin'"),
        letsPlot(df.toMap()) { x = "hwy" } + geomLollipop(stat = Stat.density(n=30)) + ggtitle("stat='density'")
    )
)

<a id="ggmarginal"></a>

#### 6. Lollipops in Marginal Layer

In [13]:
letsPlot(df.toMap()) { x = "hwy"; y = "cty" } +
    geomBin2D(binWidth = 1 to 1) +
    ggmarginal("r", size=.2,
               layer = geomLollipop(stat = Stat.count(), orientation = "y", size = 1) { color = "..count.." } )

<a id="slope_and_intercept"></a>

#### 7. Lollipops and a Regression Line

In [14]:
val xs = df["hwy"].toList().map { (it as Number).toDouble() }
val ys = df["cty"].toList().map { (it as Number).toDouble() }

fun linearModel(x: Double) = LinearRegression.fit(xs, ys, 0.95)!!.evalX(x).y

val intercept = linearModel(0.0)
val slope = linearModel(1.0) - intercept

letsPlot(df.toMap()) { x = "hwy"; y = "cty" } +
    geomSmooth(level = 0.99, seed = 42) + 
    geomLollipop(slope = slope, intercept = intercept,
                 size = 1.2, shape = 21, color = "black", fill = "magenta") +
    coordFixed()